In [2]:
!pip install google

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from googlesearch import search

In [ ]:
def sanitize_for_excel(text):
    """
    특수문자, 제어문자, 이모지 제거
    """
    text = re.sub(r'[\[\]\*\?\/\\:]', '', text)
    text = ''.join(ch for ch in text if ch.isprintable())
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text.strip())

def match_advanced_filter(text, include_keywords, exclude_keywords):
    """
    본문에 모든 포함 키워드가 존재하고 제외 키워드가 없는지 확인 (대소문자 무시)
    """
    text = text.lower()
    return all(kw.lower() in text for kw in include_keywords) and not any(bad_kw.lower() in text for bad_kw in exclude_keywords)

def crawl_tistory_via_google(keyword, include_keywords, exclude_keywords, max_results=10):
    """
    구글 검색을 활용하여 티스토리 URL들을 수집하고,
    해당 페이지에서 본문을 추출한 후 필터링한 결과를 반환
    """
    query = f"site:tistory.com {keyword}"
    results = []
    headers = {"User-Agent": "Mozilla/5.0"}
    
    for url in search(query, stop=max_results):
        try:
            res = requests.get(url, headers=headers, timeout=5)
            soup = BeautifulSoup(res.text, 'html.parser')
            title_tag = soup.find("title")
            title = title_tag.text if title_tag else url

            # 티스토리 게시글 본문 추출을 위한 후보 CSS 선택자 목록
            candidates = [
                "div.entry-content",
                "div.article",
                "div.tt_article_useless_p_margin",
                "div.contents_style",
                "div#content",
                "div.blogview_content"
            ]
            body = ""
            for sel in candidates:
                el = soup.select_one(sel)
                if el:
                    body = sanitize_for_excel(el.get_text(separator="\n"))
                    break

            if match_advanced_filter(body, include_keywords, exclude_keywords):
                results.append(("티스토리", title, url, body))
        except Exception as e:
            print("⚠️ 티스토리 오류:", e)
            continue

    return results

def save_to_file(data, base_filename="tistory_result"):
    """
    결과 데이터를 CSV 파일로 저장
    """
    df = pd.DataFrame(data, columns=["플랫폼", "제목", "링크", "본문"])
    df.to_csv(f"{base_filename}.csv", index=False, encoding='utf-8-sig')
    print(f"\n📁 저장 완료: {base_filename}.csv")

def main():
    search_keyword = input("검색할 키워드를 입력하세요 (쉼표로 구분): ").strip().split(',')
    include_keywords = input("본문에 반드시 포함할 키워드 (쉼표로 구분): ").strip().split(',')
    exclude_keywords = input("본문에 제외할 키워드 (쉼표로 구분, 없으면 Enter): ").strip().split(',')
    if exclude_keywords == ['']:
        exclude_keywords = []
    
    print("\n[🔍 티스토리 크롤링 시작...]\n")
    results = crawl_tistory_via_google(search_keyword, include_keywords, exclude_keywords)
    print(f"\n✅ 총 {len(results)}개 결과 수집됨!\n")
    
    for platform, title, link, _ in results:
        print(f"[{platform}] {title} → {link}")
    
    save_to_file(results)

if __name__ == "__main__":
    main()

▶ '부트캠프' 크롤링 시작...

✅ 총 0개 결과 수집됨!


📁 저장 완료: naver_blog_direct.csv
